In [3]:
# !pip install scikit-learn

In [4]:
import os
import pymysql
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
import json


In [5]:
db = pymysql.connect(
    # host = "aws db 엔드포인트",
    host="",
    port=3306,
    user='admin',
    # passwd='관리자 계정 비밀번호',
    passwd="",
    db='movie_db',
    charset='utf8'
)

In [6]:
sql = "select * from movie"

movie_df = pd.read_sql(sql, db)
# movie_df 
## -> synopsis_vector 컬럼에 값이 추가된 상황

C:\Users\dlsrb\AppData\Local\Temp\ipykernel_20296\3393629625.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(sql, db)


In [7]:
# 범죄도시3 wordvector를 문자열로 변환한 synopsis_vector 컬럼값을 string_array에 할당
string_array = movie_df.loc[0,"synopsis_vector"]
string_array

b"$\x0c\xd8=\x99\xdb\xf5\xbcv\x89\xa2<\xcc\x8b\xf8=\x96\xeeW\xbd\n\xf5\xaa;^\xf4\xfb;\xae5N\xbd\xa5\x87\xf9\xbd\xc1pV=\x99\xbe\x88\xbd3\xc4)\xbe\xcb\xc6\xb7\xbd\xf5\x8a\xbd<G0<>s\xcc4>\xd2D\x87\xbe\xeci\xe2\xbc\x9f\xfb-=\xdb U\xbe~\xc1:=\xd0\xe1\xf0=\x90\xdb\xb8=\xf2WW\xbd\x816N<-\xf8\xff\xbb\xad\xe9\xa2=>`\\>~\xbc5\xbe\x11\xe9e\xbe\xbc\xe4Y>O\xa5\x90\xbd\x13f\x19>\x99\x10\x95\xbdN\xca\xc0=F\xb1x>-$\x9e>\xfb\x82^\xbd\xb6\x97,\xbe\x862\xac\xbdD'\xc1\xbc\xf4\x88\x0e>\x10D]\xbe\xd12\x14< \xd4\x8d>\xf2\xb6A=1\x03\x18\xbe\xf1\xc3\x83>W\xb1p=\x82\xde\x9b=K\x97\xeb=\xb9$\xba=\xab\x81\xd7=@\x16\xf4\xbdkQ\xa7=\x1e]\x19=w}\x06>\xa14\x10\xbe\x03\xd9b\xbd\x1e\x1c^\xbev\x81\n\xbda^\x8a\xbd'\xed1\xbe\xa2\x83\x03=0\xbe\xa5\xbe\xdd\xffI\xbe\xa7\x87\xbf\xbd\x04\xd0i>\x9b\x0e1>\xba3\x81>\xe3>(=\x9d\x87\x14\xbe\xa1W\x16\xbe\xee\x8a\x89=\xd9U\x1e\xbd\x9e~M=\xc7LD<\x1f\x03\x83\xbe\xf9b\x87\xbd\x87\xd1n\xbe"

In [8]:
# 다시 numpy 배열로 반환
numpy_array = np.fromstring(string_array, dtype="float32")
# numpy_array

C:\Users\dlsrb\AppData\Local\Temp\ipykernel_20296\714448942.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  numpy_array = np.fromstring(string_array, dtype="float32")


In [20]:
# 모든 행의 synopsis_vector 컬럼 값 출력
# movie_df.loc[:,"synopsis_vector"]


In [9]:

# 모든 행의 synopsis_vector 컬럼 값을 numpy 배열로 반환
movie_df.loc[:,"synopsis_vector_numpy"] = movie_df.loc[:,"synopsis_vector"].apply(lambda x:np.fromstring(x,dtype="float32"))

C:\Users\dlsrb\AppData\Local\Temp\ipykernel_20296\1359174774.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  movie_df.loc[:,"synopsis_vector_numpy"] = movie_df.loc[:,"synopsis_vector"].apply(lambda x:np.fromstring(x,dtype="float32"))


In [25]:
# movie_df.loc[:,"synopsis_vector_numpy"]
# movie_df

In [10]:
# synopsis_vector_numpy 컬럼을 리스트로 변환
movie_df["synopsis_vector_numpy"].tolist()

## 이를 2차원 배열로 변경
np.array(movie_df["synopsis_vector_numpy"].tolist())

array([[ 0.10549191, -0.03001194,  0.01984094, ..., -0.2558832 ,
        -0.06610674, -0.23322116],
       [ 0.10602392, -0.02949523,  0.01761587, ..., -0.25908002,
        -0.06463878, -0.23570916],
       [ 0.09180073, -0.02679259,  0.01769824, ..., -0.2296537 ,
        -0.06259017, -0.21002947],
       ...,
       [ 0.11019026, -0.02762597,  0.0196363 , ..., -0.25955552,
        -0.0693134 , -0.24230015],
       [ 0.10236674, -0.03028922,  0.0188951 , ..., -0.2553198 ,
        -0.06475563, -0.22872649],
       [ 0.091694  , -0.02676833,  0.01600216, ..., -0.22744277,
        -0.05735921, -0.20284282]], dtype=float32)

#### 데이터의 각 `column` 의 평균을 뺀 다음 표준편차로 나눠 평균을 0으로 표준편차를 1로 변환하는 `StandardScalar` 객체 생성

In [11]:
scaler = StandardScaler()

In [12]:
scaler.fit(np.array(movie_df["synopsis_vector_numpy"].tolist()))

StandardScaler()

In [56]:
# scaler.mean_, scaler.var_

In [13]:
scaler.transform(np.array(movie_df["synopsis_vector_numpy"].tolist()))

array([[ 4.1885357e-02, -7.6648023e-04, -2.2218759e-01, ...,
         8.3715968e-02,  1.1782483e-01,  5.2837949e-02],
       [ 8.7208994e-02,  1.2752692e-01, -9.3090236e-01, ...,
        -2.8686492e-02,  3.0170104e-01, -3.9153248e-02],
       [-1.1245131e+00,  7.9856372e-01, -9.0466577e-01, ...,
         1.0059589e+00,  5.5830753e-01,  9.1032571e-01],
       ...,
       [ 4.4215462e-01,  5.9164488e-01, -2.8736815e-01, ...,
        -4.5405190e-02, -2.8383827e-01, -2.8284809e-01],
       [-2.2435844e-01, -6.9612399e-02, -5.2345037e-01, ...,
         1.0352483e-01,  2.8706387e-01,  2.1902342e-01],
       [-1.1336058e+00,  8.0458790e-01, -1.4448930e+00, ...,
         1.0836964e+00,  1.2135342e+00,  1.1760443e+00]], dtype=float32)

In [14]:
movie_df["synopsis_vector_numpy_scale"] = scaler.transform(np.array(movie_df["synopsis_vector_numpy"].tolist())).tolist()

In [36]:
# movie_df["synopsis_vector_numpy_scale"]
# movie_df

In [15]:

# synopsis_vector_numpy_scale 컬럼의 유클리드 거리를 계산
sim_score = euclidean_distances(movie_df["synopsis_vector_numpy_scale"].tolist(), movie_df["synopsis_vector_numpy_scale"].tolist())

sim_score

array([[0.00000000e+00, 4.31840576e+00, 9.07655060e+00, ...,
        4.82815017e+00, 2.75809504e+00, 9.81305388e+00],
       [4.31840576e+00, 1.19209290e-07, 9.50862019e+00, ...,
        4.59027313e+00, 3.32677947e+00, 1.04245588e+01],
       [9.07655060e+00, 9.50862019e+00, 2.38418579e-07, ...,
        1.06145729e+01, 8.09350928e+00, 5.09438171e+00],
       ...,
       [4.82815017e+00, 4.59027313e+00, 1.06145729e+01, ...,
        1.46000966e-07, 4.37724189e+00, 1.20509696e+01],
       [2.75809504e+00, 3.32677947e+00, 8.09350928e+00, ...,
        4.37724189e+00, 4.21468485e-08, 8.91006577e+00],
       [9.81305388e+00, 1.04245588e+01, 5.09438171e+00, ...,
        1.20509696e+01, 8.91006577e+00, 1.68587394e-07]])

In [16]:
# sim_score[0]

sim_df = pd.DataFrame(data=sim_score)
sim_df

,0,1,2,3,4,5,6,7,8,9,...,660,661,662,663,664,665,666,667,668,669
0,0.000000,4.318406e+00,9.076551e+00,1.248285e+01,1.110364e+01,11.628727,10.102911,4.676880,7.436662,12.686713,...,5.665217,7.390873,4.594519,4.325970,12.812320,20.351880,1.044449e+01,4.828150e+00,2.758095e+00,9.813054e+00
1,4.318406,1.192093e-07,9.508620e+00,1.265800e+01,1.059314e+01,12.161544,9.726583,4.282771,7.077140,13.120261,...,5.312451,8.056916,4.379233,4.009416,13.276836,20.093220,1.038310e+01,4.590273e+00,3.326779e+00,1.042456e+01
2,9.076551,9.508620e+00,2.384186e-07,6.993318e+00,1.845051e+01,4.958920,17.310476,11.214621,15.058363,5.732242,...,13.500100,4.418942,7.986378,9.369552,6.280662,27.791841,1.709808e+01,1.061457e+01,8.093509e+00,5.094382e+00
3,12.482846,1.265800e+01,6.993318e+00,4.129531e-07,2.193545e+01,5.762431,21.302442,15.175383,18.468156,5.181310,...,16.544991,6.959114,11.434824,12.601225,5.956848,31.532493,2.076945e+01,1.462125e+01,1.142822e+01,4.137918e+00
4,11.103639,1.059314e+01,1.845051e+01,2.193545e+01,2.384186e-07,21.693195,3.694471,8.437042,7.104960,22.826693,...,7.063689,16.765985,12.533565,11.359365,22.879337,10.666316,5.847136e+00,1.007164e+01,1.174982e+01,1.974208e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,20.351880,2.009322e+01,2.779184e+01,3.153249e+01,1.066632e+01,31.282903,11.110087,17.900927,15.459547,32.368644,...,16.170556,26.241659,22.095975,20.307125,32.345130,0.000000,1.149617e+01,1.876209e+01,2.121472e+01,2.931815e+01
666,10.444491,1.038310e+01,1.709808e+01,2.076945e+01,5.847136e+00,20.595683,4.349515,8.559879,8.064252,21.660246,...,7.280632,15.574524,12.068821,10.267947,21.692819,11.496172,2.384186e-07,9.701709e+00,1.126638e+01,1.866778e+01
667,4.828150,4.590273e+00,1.061457e+01,1.462125e+01,1.007164e+01,13.507204,8.852437,4.414534,6.374316,14.517661,...,5.638528,9.589182,6.425279,5.271960,14.555270,18.762088,9.701709e+00,1.460010e-07,4.377242e+00,1.205097e+01
668,2.758095,3.326779e+00,8.093509e+00,1.142822e+01,1.174982e+01,10.698146,10.814587,5.038258,8.139135,11.687715,...,6.264362,6.675044,4.157410,4.096098,11.836407,21.214720,1.126638e+01,4.377242e+00,4.214685e-08,8.910066e+00


In [17]:
sim_df.index = movie_df["title"]
sim_df.columns = movie_df["title"]

# sim_df

In [25]:
# 영화 범죄도시3 와 거리가 가까운 순으로 TOP 5 조회
## 자기 자신 제외

sim_df["범죄도시3"].sort_values()[1:6]

title
악당들                     2.758095
보이스                     2.967072
유령                      3.116467
신은 죽지 않았다 3: 어둠 속의 빛    3.117305
극한직업                    3.152396
Name: 범죄도시3, dtype: float64

In [19]:
pd.DataFrame(sim_df["범죄도시3"].sort_values()[1:6]).reset_index()

,title,범죄도시3
0,악당들,2.758095
1,보이스,2.967072
2,유령,3.116467
3,신은 죽지 않았다 3: 어둠 속의 빛,3.117305
4,극한직업,3.152396


In [20]:
sim_df["범죄도시3"].sort_values()[1:6].reset_index().values.tolist()

[['악당들', 2.758095040097118],
 ['보이스', 2.9670718645422993],
 ['유령', 3.116466566237993],
 ['신은 죽지 않았다 3: 어둠 속의 빛', 3.1173045629877465],
 ['극한직업', 3.1523961982198663]]

In [21]:
# 위의 영화 범죄도시3 와 거리가 가까운 순으로 자기 자신을 제외한 TOP 5 값을
## DataFrame 으로 바꾸고 인덱스를 지우고 해당 영화 제목을 넣은 값에서
### numpy 배열 -> 리스트로 바꾼 후 다시 DataFrame에 담아 json 문자열로 변환한 결과

result = pd.DataFrame(sim_df["범죄도시3"].sort_values()[1:6]).reset_index().values.tolist()

json.dumps(result, ensure_ascii=False)

'[["악당들", 2.758095040097118], ["보이스", 2.9670718645422993], ["유령", 3.116466566237993], ["신은 죽지 않았다 3: 어둠 속의 빛", 3.1173045629877465], ["극한직업", 3.1523961982198663]]'

In [ ]:
db.close()